<a href="https://colab.research.google.com/github/Olfeng-xalaz/Fallstudie_SCM/blob/Test1/Fallstudie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
# pip als Paketmanager
! pip install -q pyscipopt
! pip install pandas

In [ ]:
import pandas as pd
import math
from pyscipopt import Model, quicksum

# Optimierungsmodell zur Elektrifizierung der Logistik

### Import der CSV Dateien

In [ ]:
import os

# Prüfen ob in Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # In Colab: Repository klonen (nur einmal)
    if not os.path.exists('Fallstudie_SCM'):
        !git clone https://github.com/Olfeng-xalaz/Fallstudie_SCM.git
    folder = "Fallstudie_SCM/DataCSV"
else:
    # Lokal in VS Code: Relativer Pfad
    folder = os.path.join(os.path.dirname(__file__), "DataCSV") if "__file__" in dir() else "DataCSV"

In [ ]:
chargers = pd.read_csv(f"{folder}/chargers.csv", sep=";")

In [ ]:
chargers.head()

In [ ]:
dtrucks_specs = pd.read_csv(f"{folder}/diesel_trucks.csv", sep=";")

In [ ]:
dtrucks_specs.head()

In [ ]:
etrucks_specs = pd.read_csv(f"{folder}/electric_trucks.csv", sep=";")

In [ ]:
etrucks_specs.head()

In [ ]:
routes = pd.read_csv(f"{folder}/routes.csv", sep=";")

In [ ]:
routes

### Indexmengen

In [ ]:
#Erstellen einer Modellinstanz
scip = Model()

In [ ]:
R = routes["route_id"].unique() # Menge der Routen
C = chargers["charger_model"].unique() # Menge der Charger-Modelle
L = pd.concat([dtrucks_specs["truck_model"],etrucks_specs["truck_model"]]).unique() # Menge der Fahrzeugmodelle
P = range(1, 21) # Potenzielle Fahrzeuge
T = range(0, 96) # 96 Zeitintervalle pro Tag
I = range(1, 4) # Potenzielle Säulen-Slots
# T_night = [t for t in T if t > 18*4 or t <= 6*4] # Nachtintervalle: 18:00–06:00


print("R (Routen):", R)
print("C (Charger):", C)
print("L (Fahrzeugmodelle):", L)
print("P (potenzielle Fahrzeuge):", list(P))
print("T (Zeitintervalle):", list(T))
print("I (Säulen):", list(I))
# print("T_night (Nachtintervalle):", list(T_night))


### Parameter

In [ ]:
# 3) Parameter aus routes.csv
# -----------------------------
dist_total = dict(zip(routes["route_id"], routes["distance_total"]))
dist_toll  = dict(zip(routes["route_id"], routes["distance_toll"]))
start_time = dict(zip(routes["route_id"], routes["starttime"]))
end_time   = dict(zip(routes["route_id"], routes["endtime"]))

# Hinweis:
# A[r,t] (binär: Tour r läuft im Intervall t) und g[r,t] (kWh-Verbrauch pro Intervall)
# hängen von Zeitdiskretisierung + Tourdauer ab.
# Das bauen wir später, sobald klar ist, wie starttime/endtime formatiert sind.

# -----------------------------
# 4) Parameter aus chargers.csv
# -----------------------------
capex_ch = dict(zip(chargers["charger_model"], chargers["capex_yearly"]))
opex_ch  = dict(zip(chargers["charger_model"], chargers["opex_yearly"]))
pmax_ch  = dict(zip(chargers["charger_model"], chargers["max_power"]))        # kW
spots_ch = dict(zip(chargers["charger_model"], chargers["charging_spots"]))   # Anzahl Ladepunkte

# -----------------------------
# 5) Parameter aus electric_trucks.csv
# -----------------------------
capex_veh = {}
opex_veh  = {}

cons_e = {}       # kWh/100km
thg_e  = {}       # €/a
pmax_veh = {}     # kW max Ladeleistung
batt_kwh = {}     # kWh Batterie (soc_max)

for _, row in etrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    cons_e[m]    = row["avg_energy_kWh_per_100km"]
    thg_e[m]     = row["thg_yearly"]
    pmax_veh[m]  = row["max_power"]
    batt_kwh[m]  = row["soc_max_kWh"]

batt_kwh['ActrosL'] = 0
pmax_veh['ActrosL'] = 0

# -----------------------------
# 6) Parameter aus diesel_trucks.csv
# -----------------------------
kfz_d = {}       # €/a
cons_d = {}      # ggf. l/100km (nur falls du Diesel-Kraftstoffkosten modellierst)

for _, row in dtrucks_specs.iterrows():
    m = row["truck_model"]
    capex_veh[m] = row["capex_yearly"]
    opex_veh[m]  = row["opex_yearly"]
    if "kfz_yearly" in dtrucks_specs.columns:
        kfz_d[m] = row["kfz_yearly"]
    if "avg_diesel_per_100km" in dtrucks_specs.columns:
        cons_d[m] = row["avg_diesel_per_100km"]

cons_e['ActrosL'] = 0
print(cons_e)

# -----------------------------
# 7) Abgeleitete Parameter: Energiebedarf pro Route und e-Lkw (E[r,e])
# -----------------------------
E_route_e = {}  # (r,e) -> kWh

for r in R:
    for e in cons_e.keys():
        E_route_e[(r, e)] = dist_total[r] * cons_e[e] / 100.0

# -----------------------------
# 8) Falltext-Parameter (Konstanten)
# -----------------------------
N_days = 260
delta_h = 0.25  # 15 Minuten = 0.25 Stunden
diesel = 1.60
SOC_T_Start = 0

# Netz & Tarif
P_grid_max = 500.0 # max kW am Depot
P_grid_add = 500.0 # Zusatzleistung bei Netzausbau
capex_grid_add = 10000.0 # Kosten/Jahr für Netzausbau

c_energy = 0.25   # Arbeitspreis Strom in €/kWh
c_capex = 1000.0   # Stromkosten in €/a
c_peak = 150.0    # Leistungspreis in €/kW

# Maut
c_toll = 0.34      # €/km mautpflichtig

# Speicher
c_capex_bat_kW = 30.0      # Batteriekosten in €/kW
c_capex_bat_kWh = 350.0     # Batteriekosten in €/kWh
roundtrip_eff = 0.98 # Round-Trip Efficiency aka Wirkungsgrad
eta = math.sqrt(roundtrip_eff)  # für lineare Lade/Entlade-Gleichungen
dod = 0.975 # Max. Entladetiefe
soc_bat_min_frac = 1.0 - dod    # = 0.025


# -----------------------------
# 9) Kurzer Test-Print der wichtigsten Parameter
# -----------------------------
print("Beispiel dist_total[r]:", list(dist_total.items())[:3]); print()
print("Charger pmax (kW):", pmax_ch); print()
print("e-Lkw batt_kwh:", batt_kwh); print()
print("Energiebedarf Beispiel (erste Route, e400/e600):")
first_r = R[0]
for e in cons_e.keys():
    print(" ", (first_r, e), "=", E_route_e[(first_r, e)], "kWh")
print(E_route_e)

print("Konstanten: P_grid_max =", P_grid_max, "| c_energy =", c_energy, "| c_toll =", c_toll); print()
print("Speicher: eta =", eta, "| soc_bat_min_frac =", soc_bat_min_frac); print()

## Hilfsfunktionen

In [ ]:
def time_to_t_interval(uhrzeit):
  # Den String am Doppelpunkt trennen
  stunden_str, minuten_str = uhrzeit.split(":")
  # In Zahlen (Integer) umwandeln
  stunden = int(stunden_str)
  minuten = int(minuten_str)
  t = stunden * 4
  t = t + minuten / 15
  return t

## Weitere Parameter

In [ ]:
# Verbrauch pro Zeitintervall pro LKW Art
# Erweitere deine Verbrauchsmatrix
verbrauch_pro_intervall = {}
for r in R:
    # 1. Daten holen (aus deinen Routen-Daten)
    dist = dist_total[r]
    start = time_to_t_interval(start_time[r])
    ende = time_to_t_interval(end_time[r])
    dauer = ende - start

    for l in L:
        # 2. Verbrauch pro Intervall berechnen
        if l in ['eActros400', 'eActros600']:
            # Elektro: Distanz * Verbrauch / Dauer
            #print(dist, cons_e[l], dauer)
            kwh_pro_t = (dist/100 * cons_e[l]) / dauer
        else:
            # Diesel: Verbraucht 0 kWh Strom
            kwh_pro_t = 0

        # 3. In Matrix speichern
        for t in T:
            if start <= t < ende:
                verbrauch_pro_intervall[r, l, t] = kwh_pro_t
            else:
                verbrauch_pro_intervall[r, l, t] = 0

In [ ]:
route_aktiv = {}
for r in R:
    start_t = time_to_t_interval(start_time[r])
    ende_t = time_to_t_interval(end_time[r])
    for t in T:
        if start_t <= t < ende_t:
            route_aktiv[r, t] = 1
        else:
            route_aktiv[r, t] = 0

### Erweiterung: Parameter für die PV-Anlage

In [ ]:
pv_kw = 200 # Maximal mögliche installierte Leistung in KW von der PV-Anlage
capex_pv_yearly = 10000  # Jährliche Investitionskosten der PV-Anlage
opex_pv_yearly = 2000    # Jährliche Betriebskosten der PV-Anlage

pv_profil = {}
for t in T:
    # Die Sonne scheint typischerweise zwischen 06:00 (t=24) und 18:00 (t=72)
    if 24 <= t <= 72:
        # Faktor f_t: Sinus-Kurve simuliert den Sonnenstand (Glockenkurve)
        # Erreicht das Maximum (1.0) genau um 12:00 Uhr (t=48)
        f_t = math.sin(math.pi * (t - 24) / 48)
        pv_profil[t] = pv_kw * f_t
    else:
        pv_profil[t] = 0

c_pv_fix = (capex_pv_yearly  + opex_pv_yearly) / 260 # Fixkosten der PV-Anlage
# Keine variablen Kosten, Rohstoff "Sonne ist kostenlos" !!!!

## Entscheidungsvariablen

In [ ]:
Flottenwahl = {}
for l in L:
    for p in P:
        Flottenwahl[l, p] = scip.addVar(vtype = 'B', name =f"LKW_{l}_auf_ID_{p}")

Zuordnung_LKW_Route = {}
for r in R:
    for p in P:
        Zuordnung_LKW_Route[r, p] = scip.addVar(vtype = 'B', name =f"Route_{r}_wird_von_LKW_mit_ID_{p}_gefahren")

Auswahl_Charger = {}
for c in C:
    for i in I:
        Auswahl_Charger[c, i] = scip.addVar(vtype = 'B', name =f"Charger_{c}_ist_Säule_Nummer_{i}")

Zuordnung_LKW_Zeitpunkt_Charger = {}
for p in P:
    for t in T:
        for i in I:
            Zuordnung_LKW_Zeitpunkt_Charger[p,t,i] = scip.addVar(vtype = 'B', name =f"LKW_{l}_zum_Zeitpunkt_{t}_an_Charger_{c}")

SOC_Zeit_Fahrzeug = {}
Ladeleistung = {} # Von p verwendete Ladeleistung zum Zeitpunkt t
for p in P:
  for t in T:
    SOC_Zeit_Fahrzeug[p,t] = scip.addVar(vtype = 'C',lb=0, name=f"SOC_FahrzeugID_{p}_Zeitpunkt{t}")
    Ladeleistung[p, t] = scip.addVar(vtype='C', lb=0, name=f"Ladeleistung_{p}_{t}")

# Batteriespeicher
SOC_Speicher = {}      # Aktueller Stand in kWh
Laden_Speicher = {}    # Leistung vom Netz in den Speicher (kW)
Entladen_Speicher = {} # Leistung vom Speicher ins Depot/Lkw (kW)

for t in T:
    SOC_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"SOC_Speicher_t{t}")
    Laden_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"Laden_Speicher_t{t}")
    Entladen_Speicher[t] = scip.addVar(vtype='C', lb=0, name=f"Entladen_Speicher_t{t}")


Einsatz = {}
for r in R:
    for l in L:
        for p in P:
            # Diese Variable ist 1, wenn ID p Route r fährt UND Typ l ist
            Einsatz[r, l, p] = scip.addVar(vtype='B', name=f"Einsatz_{r}_{l}_{p}")


Ausbau_Netz = scip.addVar(vtype='B', name="Netzausbau")


Batterie_kWh = scip.addVar(vtype='C', lb=0, name="Batteriekapa_kWh")
Batterie_kW = scip.addVar(vtype='C', lb=0, name="Batterieleistung_kW")

P_peak = scip.addVar(vtype='C', lb=0, name="Jahres_Leistungsspitze_kW")

E_LKWs_in_Flotte = scip.addVar(vtype="B", name="E_LKWs_in_Flotte")

# Hilfsvariable zur korrekten Verteilung der Leistung auf die Säulen
Ladeleistung_Saeule = {}
for p in P:
    for t in T:
        for i in I:
            Ladeleistung_Saeule[p, t, i] = scip.addVar(lb=0, ub=500, name=f"P_S_{p}_{t}_{i}")

len(scip.getVars())



### Erweiterung: Entscheidungsvariable

In [ ]:
# Hilfsvariable für den Netzbezug in kw aus dem Stromnetzt, der durch PV reduziert wird, um gleichzeitg fiktive Gewinne durch die Sonne zu vermeiden
netzbezug = {}
for t in T:
    netzbezug [t] = scip.addVar(lb=0, name=f"P_Netz_t{t}") #lb = 0 sehr wichtig

## Restriktionen

In [ ]:
#Für Einsatz{}
for r in R:
    for l in L:
        for p in P:
            # Wenn Einsatz=1, MUSS Zuordnung=1 sein
            scip.addCons(Einsatz[r, l, p] <= Zuordnung_LKW_Route[r, p])
            # Wenn Einsatz=1, MUSS Flottenwahl=1 sein
            scip.addCons(Einsatz[r, l, p] <= Flottenwahl[l, p])
            # Wenn beide 1 sind, MUSS Einsatz 1 werden
            scip.addCons(Einsatz[r, l, p] >= Zuordnung_LKW_Route[r, p] + Flottenwahl[l, p] - 1)
print("1.")
print (scip.getNConss())

# Zuordnung_LKW_Route{}
#1. Jede Route muss genau einmal bedient werden
for r in R:
  scip.addCons(quicksum(Zuordnung_LKW_Route[r,p] for p in P)==1)


#2. Jede Route r braucht ein Fahrzeug p, und dieses Fahrzeug p MUSS ein Modell l haben
for r in R:
    for p in P:
        # Wenn Route r dem Fahrzeug p zugeordnet ist (Variable == 1),
        # dann muss die Summe der Flottenwahl über alle Modelle l für dieses p ebenfalls 1 sein.
        scip.addCons(
            Zuordnung_LKW_Route[r, p] <= quicksum(Flottenwahl[l, p] for l in L),
            name=f"Route_braucht_Modell_{r}_{p}"
        )

print("2.")
print (scip.getNConss())
# Flottenwahl{}
#1. Max 20 Fahrzeuge
scip.addCons(quicksum(Flottenwahl[l,p] for p in P for l in L)<=20)

#2. Jede ID hat 1 oder weniger(0) LKW Modelle
for p in P:
  scip.addCons(quicksum(Flottenwahl[l,p] for l in L)<=1)

print("3.")
print (scip.getNConss())
# SOC_Zeit_Fahrzeug{}
#1. Berechnung SOC zum Zeitpunkt t
for p in P:
    for t in T:
        if t == 0:
            # Setzt den SOC zum Zeitpunkt 0 fest auf SOC_T_Start
            scip.addCons(SOC_Zeit_Fahrzeug[p, t] == SOC_T_Start, name=f"Start_SOC_{p}")
        else:
            # Für alle anderen Zeitpunkte gilt die normale Bilanz
            t_prev = t - 1
            energie_geladen = Ladeleistung[p, t] * 0.25
            energie_verbrauch = quicksum(Einsatz[r, l, p] * verbrauch_pro_intervall[r, l, t] for r in R for l in L)

            scip.addCons(SOC_Zeit_Fahrzeug[p, t] == SOC_Zeit_Fahrzeug[p, t_prev] + energie_geladen - energie_verbrauch,
                         name=f"SOC_Update_{p}_{t}")

#2. Regel dass SOC am Anfang des Tages[t=0] = SOC Ende des Tages[t=95] sein muss
for p in P:
  scip.addCons(SOC_Zeit_Fahrzeug[p,0] == SOC_Zeit_Fahrzeug[p, 95])

#3. SOC <= Max.Kappa LKW
for p in P:
    for t in T:
        # Der SOC von LKW p zum Zeitpunkt t darf die Kapazität
        # des gewählten Modells l nicht überschreiten.
        scip.addCons(
            SOC_Zeit_Fahrzeug[p, t] <= quicksum(Flottenwahl[l, p] * batt_kwh[l] for l in L),
            name=f"Max_Kapazitaet_LKW_{p}_t{t}"
        )

# 4. SOC niemals < 0 bereits über lower bound definiert


# Route_Aktiv{}
# 1. Keine_Ueberlappung
for p in P:
    for t in T:
        # Die Summe aller Routen r, die zum Zeitpunkt t aktiv sind,
        # darf für LKW p nicht größer als 1 sein.
        scip.addCons(
            quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R) <= 1,
            name=f"Keine_Ueberlappung_LKW_{p}_t{t}"
        )
print("4.")
print (scip.getNConss())
# Ladeleistung [p,t]
# 1. Nur Laden wenn am Depot / nicht auf Tour
for p in P:
    for t in T:
        # Summe ist 1, wenn der LKW fährt, 0 wenn er im Depot ist
        ist_auf_tour = quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

        # Wenn ist_auf_tour == 1, muss Ladeleistung <= 0 sein (also 0)
        # Wenn ist_auf_tour == 0, darf er laden (begrenzt durch einen großen Wert/M-Faktor)
        scip.addCons(Ladeleistung[p, t] <= (1 - ist_auf_tour) * max(pmax_veh.values()),
                     name=f"Laden_nur_im_Depot_{p}_{t}")

# 3. Max_Power von LKW-Model
for p in P:
    for t in T:
        # Die Ladeleistung darf die fahrzeugspezifische Grenze nicht überschreiten
        scip.addCons(
            Ladeleistung[p, t] <= quicksum(Flottenwahl[l, p] * pmax_veh[l] for l in L),
            name=f"Max_Ladeleistung_Fahrzeug_{p}_{t}"
        )
# --- KORREKTUR: Ladeleistung & Säulenkapazität ---

for p in P:
    for t in T:
        # A. Leistungsbilanz: Summe der Säulenleistung = Gesamtleistung des LKW
        scip.addCons(
            quicksum(Ladeleistung_Saeule[p, t, i] for i in I) == Ladeleistung[p, t],
            name=f"Leistungsbilanz_{p}_{t}"
        )

        for i in I:
            # B. Kopplung: Leistung kann nur fließen, wenn Stecker in Säule i steckt
            scip.addCons(
                Ladeleistung_Saeule[p, t, i] <= max(pmax_veh.values()) * Zuordnung_LKW_Zeitpunkt_Charger[p, t, i],
                name=f"Leistungs_Kopplung_{p}_{t}_{i}"
            )

# C. Max Leistung je Charger (Echte Kapazität pro Säule)
for i in I:
    for t in T:
        scip.addCons(
            quicksum(Ladeleistung_Saeule[p, t, i] for p in P) <=
            quicksum(Auswahl_Charger[c, i] * pmax_ch[c] for c in C),
            name=f"Echte_Saeulenkapazitaet_i{i}_t{t}"
        )

# D. Netzanschluss-Limit (Jetzt mit korrekter Summe) --> Wird bei der Erweiterung definiert
#for t in T:
#   # Hier nutzen wir die Ladeleistung[p,t], da sie nun korrekt durch die Säulen limitiert wird
#    netzbezug = quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]
#    scip.addCons(netzbezug <= 500 + (500 * Ausbau_Netz), name=f"Netz_Limit_t{t}")
#    scip.addCons(netzbezug >= 0, name=f"Keine_Rückspeisung_t{t}")


print("5.")
print (scip.getNConss())
#Zuordnung_LKW_Zeitpunkt_Charger[p,t,i]
#1. Max 2 LKW pro Säule und Säule muss existieren
for i in I:
    for t in T:
        scip.addCons(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for p in P)
            <= quicksum(spots_ch[c] * Auswahl_Charger[c, i] for c in C),
            name=f"Spots_Limit_Saeule_{i}_t{t}"
        )
#2. Ein LKW an max 1 Säule laden
for p in P:
    for t in T:
        # Ein LKW p kann zum Zeitpunkt t an maximal einer Säule i hängen
        scip.addCons(
            quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i] for i in I) <= 1,
            name=f"Max_Eine_Saeule_Pro_LKW_{p}_{t}"
        )

# 3.Säule nicht wechseln
for p in P:
    for t in T:
        if t < 95: # Nicht für das letzte Intervall (t+1 wäre sonst out of range)
            for i in I:
                # Logik: Wenn (LKW an Säule i zu t) UND (LKW lädt zu t+1),
                # dann MUSS (LKW an Säule i zu t+1)
                # Mathematisch über eine lineare Bedingung:
                scip.addCons(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + quicksum(Zuordnung_LKW_Zeitpunkt_Charger[p, t+1, j] for j in I if j != i)
                    <= 1,
                    name=f"Kein_Saeulenwechsel_{p}_{t}_{i}"
                )

# 4.
# Definition der Nachtzeit (18:00 bis 06:00 Uhr)
# 18:00 Uhr ist Intervall 72, 06:00 Uhr ist Intervall 24
T_Nacht = [t for t in T if t >= 72 or t < 24]

for p in P:
    for t in T_Nacht:
        t_next = (t + 1) % 96
        # Nur wenn das nächste Intervall auch noch in der Nacht liegt
        if t_next in T_Nacht:
            # Wenn der Lkw NICHT auf Tour ist (also im Depot parkt)
            ist_im_depot = 1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t] for r in R)

            for i in I:
                # Regel: Wenn er an Säule i steht, muss er dort stehen bleiben,
                # solange er im Depot ist (kein Umparken nachts).
                # Logik: Stecker(t, i) + ImDepot(t+1) - Stecker(t+1, i) <= 1
                # Das erzwingt: Wenn er zu t an i war und zu t+1 noch da ist,
                # MUSS er zu t+1 auch an i sein.
                scip.addCons(
                    Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]
                    + (1 - quicksum(Zuordnung_LKW_Route[r, p] * route_aktiv[r, t_next] for r in R))
                    - Zuordnung_LKW_Zeitpunkt_Charger[p, t_next, i] <= 1,
                    name=f"Nacht_Anschluss_Pflicht_{p}_{t}_{i}"
                )

print("6.")
print (scip.getNConss())
#Auswahl_Charger(c,i)
#1. Max 1 Charger Model pro Charger Platz
for i in I:
    # Jede Säule i (1, 2, 3) darf maximal einen Typ c haben
    scip.addCons(
        quicksum(Auswahl_Charger[c, i] for c in C) <= 1,
        name=f"Max_Ein_Typ_Pro_Saeule_{i}"
    )

print("7.")
print (scip.getNConss())
#Batteriespeicher
# 1. SOC-Berechnung
for t in T:
    t_prev = t - 1 if t > 0 else 95

    # Bilanz: Stand vorher + Laden - Entladen
    # Faktor 0.25 für 15-Min-Intervalle
    scip.addCons(
        SOC_Speicher[t] == SOC_Speicher[t_prev]
        + (Laden_Speicher[t] * 0.25 * 0.99)
        - (Entladen_Speicher[t] * 0.25 / 0.99),
        name=f"SOC_Bilanz_Speicher_t{t}"
    )

for t in T:
    # 1. Maximale Kapazität (nach oben gedeckelt durch Entscheidungsvariable)
    scip.addCons(SOC_Speicher[t] <= Batterie_kWh, name=f"Speicher_Max_Kappa_t{t}")

    # 2. Minimale Kapazität (DoD 97,5% -> 2,5% müssen drin bleiben)
    scip.addCons(SOC_Speicher[t] >= 0.025 * Batterie_kWh, name=f"Speicher_Min_DoD_t{t}")

for t in T:
    # Laden vom Netz begrenzt
    scip.addCons(Laden_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Laden_t{t}")

    # Entladen ins Depot begrenzt
    scip.addCons(Entladen_Speicher[t] <= Batterie_kW, name=f"Speicher_Max_Entladen_t{t}")

scip.addCons(SOC_Speicher[0] == SOC_Speicher[95], name="Speicher_Zyklus_Check")


# Peak Leistung --> Wird unten mit der Erweiterung kombiniert !
# 2. Constraints: P_peak muss größer sein als die Last in JEDEM Zeitintervall
# Wir gehen durch alle 96 Viertelstunden (t in T)
#for t in T:
    # Berechne die tatsächliche Netzlast in diesem Intervall:
    # Last = (Summe aller LKW-Ladeleistungen) + (Speicher laden) - (Speicher entladen)
#  aktuelle_netzlast = quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]

    # Wir fügen die Bedingung hinzu: P_peak >= aktuelle_netzlast
 #   scip.addCons(P_peak >= aktuelle_netzlast, name=f"Peak_Check_t{t}")

print("8.")


# Nur wenn mind. 1 E-LKW in flotte wird E_LKWs_in_Flotte = 1 gesetzt, für die Berechnung von c_capex
# Liste der E-Lkw Modelle (alle außer Diesel)
E_Modelle = [l for l in L if l != 'ActrosL']

# Wenn irgendein E-Lkw gewählt wird, muss die Infrastruktur aktiv sein
# Wir nutzen eine Big-M Kopplung:
# Die Summe aller gewählten E-Lkw muss kleiner gleich (Anzahl IDs * Aktiv-Variable) sein.
scip.addCons(
    quicksum(Flottenwahl[l, p] for l in E_Modelle for p in P) <= len(P) * E_LKWs_in_Flotte,
    name="E_LKWs_in_Flotte"
)


# Rechenzeit optimierung
# Symmetrie-Brechung: Verhindert, dass der Solver identische Fahrzeug-IDs vertauscht
for i in range(len(P) - 1):
    p_current = P[i]
    p_next = P[i+1]
    # "Ein Modell für Fahrzeug i+1 darf nur gewählt werden, wenn auch für Fahrzeug i eines gewählt wurde"
    scip.addCons(
        quicksum(Flottenwahl[l, p_current] for l in L) >=
        quicksum(Flottenwahl[l, p_next] for l in L),
        name=f"Symmetry_Break_{p_current}_{p_next}"
    )

# Symmetrie-Brechung Ladesäulen:
# Eine Säule i+1 darf nur gebaut werden, wenn Säule i auch gebaut ist.
# Das verhindert Lücken wie [Gebaut, Leer, Gebaut].
for i in range(len(I) - 1):
    scip.addCons(
        quicksum(Auswahl_Charger[c, I[i]] for c in C) >=
        quicksum(Auswahl_Charger[c, I[i+1]] for c in C),
        name=f"Charger_Order_{i}"
    )

scip.getNConss()

### Erweiterung: Restriktionen

In [ ]:
# D. Netzbezug mit berücksichtigung auf PV.
for t in T:
    scip.addCons(netzbezug[t] >= (quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t] - pv_profil[t]), name=f"Netzbezug_Bilanz_t{t}")
    scip.addCons(netzbezug[t] <= 500 + (500 * Ausbau_Netz), name=f"Netz_Limit_t{t}")
    scip.addCons(P_peak >= netzbezug[t], name=f"Peak_Check_t{t}") # Constraint: P_peak muss größer sein als die Last in JEDEM Zeitintervall, wir gehen durch alle 96 Viertelstunden (t in T)

## Zielfunktion

In [ ]:
# 1. Anteilige Fahrzeugkosten (pro Tag)
obj_veh = (1/260) * quicksum(
    Flottenwahl[l, p] * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L for p in P
)

# 2. Anteilige Kosten Ladeinfrastruktur (pro Tag)
obj_charger = (1/260) * (
    (E_LKWs_in_Flotte * c_capex) + # Fällt nur an, wenn E-Lkws existieren
    quicksum(Auswahl_Charger[c, i] * (capex_ch[c] + opex_ch[c]) for c in C for i in I)
)

# 3. Anteilige Kosten Speicher (pro Tag)
obj_battery = (1/260) * (
    (Batterie_kWh * c_capex_bat_kWh) +
    (Batterie_kW * c_capex_bat_kW) +
    (Batterie_kWh * 0.02 * c_capex_bat_kWh) +
    (Batterie_kW * 0.02 * c_capex_bat_kW)
)

# 4. Anteilige Kosten Netzausbau (pro Tag)
obj_grid = (1/260) * (Ausbau_Netz * capex_grid_add)

# 5. Variable Kosten (Direkt pro Tag)

# Stromkosten (LKW + Speicher Laden - Speicher Entladen) * Umrechnung 15 min Intervall * Energiepreis
#cost_electricity = quicksum(
 #  (quicksum(Ladeleistung[p, t] for p in P) + Laden_Speicher[t] - Entladen_Speicher[t]) *0.25 * c_energy
#    for t in T
#) --> Bei Erweiterung neu definiert

cost_peak = (1 / 260) * P_peak * c_peak

# Mautkosten NUR für Diesel (ActrosL)
# Wir koppeln das an die Variable Einsatz[r, l, p], da wir hier
# den LKW-Typ 'l' direkt prüfen können.
cost_toll = quicksum(
    Einsatz[r, 'ActrosL', p] * dist_toll[r] * c_toll
    for r in R for p in P if 'ActrosL' in L
)

# Dieselkosten (nur für Diesel-LKW 'ActrosL')
cost_diesel = quicksum(
    Einsatz[r, 'ActrosL', p] * (dist_total[r] * cons_d.get('ActrosL', 0) / 100.0) * diesel
    for r in R for p in P if 'ActrosL' in L
)

# GESAMT-ZIELFUNKTION --> Bei Erweiterung
#scip.setObjective(
#    obj_veh + obj_charger + obj_battery + obj_grid + cost_electricity + c_peak + cost_toll + cost_diesel,
#    "minimize"
#)

### Erweiterung Zielfunktion

In [ ]:
# Stromkosten (LKW + Speicher Laden - Speicher Entladen) = Netzbezug [t] * Umrechnung 15 min Intervall * Energiepreis
cost_electricity = quicksum(netzbezug[t] * 0.25 * c_energy for t in T)

# GESAMT-ZIELFUNKTION + Fixe Kosten der PV-Anlage
scip.setObjective(
    obj_veh + obj_charger + obj_battery + obj_grid + cost_electricity + cost_peak + cost_toll + cost_diesel + c_pv_fix,
    "minimize"
)

## Berechnung des Ergebnisses

In [ ]:
#scip.setRealParam("limits/time", 30)
scip.redirectOutput()
scip.optimize()

presolving:
(round 1, fast)       461 del vars, 1644 del conss, 0 add conss, 9676 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 2, fast)       642 del vars, 1911 del conss, 0 add conss, 9937 chg bounds, 0 chg sides, 200 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 3, fast)       822 del vars, 2177 del conss, 0 add conss, 10117 chg bounds, 0 chg sides, 200 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 4, exhaustive) 822 del vars, 3242 del conss, 0 add conss, 10237 chg bounds, 20 chg sides, 400 chg coeffs, 0 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 5, exhaustive) 822 del vars, 3242 del conss, 0 add conss, 10277 chg bounds, 20 chg sides, 480 chg coeffs, 20627 upgd conss, 1380 impls, 11943 clqs, 0 implints
(round 6, medium)     822 del vars, 3263 del conss, 38 add conss, 10297 chg bounds, 58 chg sides, 614 chg coeffs, 20627 upgd conss, 6936 impls, 12000 clqs, 0 implints
(round 7, exha

In [ ]:
print ("Zielfunktions Wert Kosten: ",scip.getObjVal())

## Solution Output

In [ ]:
print("--- Fahrzeugmodell-Zuordnung ---")
for l in L:
    for p in P:
        if scip.getVal(Flottenwahl[l, p]) > 0.5:
            print(f"Fahrzeug ID {p}: Modell {l}")

print("E_LKWs_in_Flotte Ja/Nein")
if scip.getVal(E_LKWs_in_Flotte) > 0.5:
    print("Ja")
else:
    print("Nein")


print("\n--- Optimierte Ladeergebnisse ---")
for t in T:
    # Checke, ob in diesem Intervall überhaupt etwas passiert
    if any(scip.getVal(Ladeleistung[p, t]) > 0.1 for p in P):
        print(f"Zeitintervall {t}:")
        for p in P:
            p_val = scip.getVal(Ladeleistung[p, t])
            if p_val > 0.1:
                # Finde die Säule
                saeule = next((i for i in I if scip.getVal(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]) > 0.5), "Keine")
                print(f"  - LKW {p} lädt {p_val:.1f} kW an Säule {saeule}")

print("\n--- Routen-Zuordnung (Route Assignment) ---")
for r in R:
    for p in P:
        if scip.getVal(Zuordnung_LKW_Route[r, p]) > 0.5:
            print(f"Route {r}: Gefahren von Fahrzeug ID {p}")

print("\n--- Auswahl Ladegeräte (Charger Selection) ---")
for c in C:
    for i in I:
        if scip.getVal(Auswahl_Charger[c, i]) > 0.5:
            print(f"Ladeplatz {i}: Modell {c}")

print("--- SOC für Fahrzeuge wo SOC >0 ---")
for p in P:
    for t in T:
        soc_val = scip.getVal(SOC_Zeit_Fahrzeug[p, t])
        if soc_val > 0:
          print(f"  Zeitintervall {t}; Fahrzeug: {p}; SOC={soc_val:.2f} kWh")

print("\n--- Ladeleistung----")
for p in P:
  for t in T:
    ladeleistung_val = scip.getVal(Ladeleistung[p, t])
    if ladeleistung_val > 0:
      print(f"Fahrzeug ID {p}, Zeitintervall {t}: Ladeleistung={ladeleistung_val:.2f} kW")

print("\n--- Netzausbau (Grid Expansion) ---")
if scip.getVal(Ausbau_Netz) > 0.5:
    print("Netzausbau: Ja")
else:
    print("Netzausbau: Nein")

if (scip.getVal(Batterie_kWh) > 0.5) or (scip.getVal(Batterie_kW) > 0.5):
    print("\n--- Batteriespeicher (Battery Storage) ---")
    print(f"Batteriespeicher gekauft: Ja")
    print(f"  Kapazität: {scip.getVal(Batterie_kWh):.2f} kWh")
    print(f"  Leistung: {scip.getVal(Batterie_kW):.2f} kW")
else:
    print("Batteriespeicher gekauft: Nein")


print("\n--- SOC und Laden/Entladen des Speichers (beispielhaft für die ersten 24 Zeitintervalle) ---")
for t in range(0, 24): # Display first 24 time intervals (6 hours)
    soc_storage = scip.getVal(SOC_Speicher[t])
    charge_storage = scip.getVal(Laden_Speicher[t])
    discharge_storage = scip.getVal(Entladen_Speicher[t])
    if soc_storage > 0:
      print(f"  Zeitintervall {t}: SOC_Speicher={soc_storage:.2f} kWh, Laden_Speicher={charge_storage:.2f} kW, Entladen_Speicher={discharge_storage:.2f} kW")

print("\n--- Batterie ---")
print(scip.getVal(Batterie_kWh))
print(scip.getVal(Batterie_kW))

### Erweiterung: Solution Output

In [ ]:
if scip.getStatus() == "optimal":
    print("\n--- PV-ANLAGEN ANALYSE ---")
    tages_ertrag_pv = sum(pv_profil[t] * 0.25 for t in T)
    print(f"Theoretischer PV-Ertrag heute: {round(tages_ertrag_pv, 2)} kWh")

In [ ]:
print("Model Fertig")

## Excel - Export

In [ ]:
# Export der Optimierungsergebnisse nach Excel
import pandas as pd

excel_path = f"optimierungsergebnisse_{scip.getGap()*100:.2f}%_Gap.xlsx"

# Flottenwahl (welches Modell auf welcher Fahrzeug-ID)
fleet_rows = []
for l in L:
    for p in P:
        fleet_rows.append({
            "truck_model": l,
            "vehicle_id": p,
            "value": float(scip.getVal(Flottenwahl[l, p]))
        })
df_fleet = pd.DataFrame(fleet_rows)

# Routen-Zuordnung (welche Route wird von welcher ID gefahren)
route_assign_rows = []
for r in R:
    for p in P:
        route_assign_rows.append({
            "route_id": r,
            "vehicle_id": p,
            "value": float(scip.getVal(Zuordnung_LKW_Route[r, p]))
        })
df_route_assign = pd.DataFrame(route_assign_rows)

# Einsatz-Variable (Route, LKW-Typ, Fahrzeug-ID)
einsatz_rows = []
for r in R:
    for l in L:
        for p in P:
            einsatz_rows.append({
                "route_id": r,
                "truck_model": l,
                "vehicle_id": p,
                "value": float(scip.getVal(Einsatz[r, l, p]))
            })
df_einsatz = pd.DataFrame(einsatz_rows)

# Auswahl der Charger-Modelle je Säule
charger_rows = []
for c in C:
    for i in I:
        charger_rows.append({
            "charger_model": c,
            "slot": i,
            "value": float(scip.getVal(Auswahl_Charger[c, i]))
        })
df_charger = pd.DataFrame(charger_rows)

# SOC der Fahrzeuge über die Zeit
soc_rows = []
for p in P:
    for t in T:
        soc_rows.append({
            "vehicle_id": p,
            "time_interval": t,
            "soc_kwh": float(scip.getVal(SOC_Zeit_Fahrzeug[p, t]))
        })
df_soc = pd.DataFrame(soc_rows)

# Ladeleistung der Fahrzeuge über die Zeit
power_rows = []
for p in P:
    for t in T:
        power_rows.append({
            "vehicle_id": p,
            "time_interval": t,
            "power_kw": float(scip.getVal(Ladeleistung[p, t]))
        })
df_power = pd.DataFrame(power_rows)

# Zuordnung LKW-Zeitpunkt-Charger
charger_assign_rows = []
for p in P:
    for t in T:
        for i in I:
            charger_assign_rows.append({
                "vehicle_id": p,
                "time_interval": t,
                "slot": i,
                "value": float(scip.getVal(Zuordnung_LKW_Zeitpunkt_Charger[p, t, i]))
            })
df_charger_assign = pd.DataFrame(charger_assign_rows)

# Speicher-Zustände und Leistungen
storage_rows = []
for t in T:
    storage_rows.append({
        "time_interval": t,
        "soc_storage_kwh": float(scip.getVal(SOC_Speicher[t])),
        "charge_kw": float(scip.getVal(Laden_Speicher[t])),
        "discharge_kw": float(scip.getVal(Entladen_Speicher[t]))
    })
df_storage = pd.DataFrame(storage_rows)

# Skalare Größen und Kennzahlen
scalar_rows = [
    {"variable": "Objektwert_gesamt", "value": float(scip.getObjVal())},
    {"variable": "Ausbau_Netz", "value": float(scip.getVal(Ausbau_Netz))},
    {"variable": "Batterie_kWh", "value": float(scip.getVal(Batterie_kWh))},
    {"variable": "Batterie_kW", "value": float(scip.getVal(Batterie_kW))},
    {"variable": "P_peak", "value": float(scip.getVal(P_peak))}
 ]
df_scalars = pd.DataFrame(scalar_rows)

# Kostenaufschlüsselung gemäß Zielfunktion (pro Tag)
# 1) Fahrzeugkosten nach E-Lkw und Verbrenner getrennt
L_e = set(etrucks_specs["truck_model"])
L_d = set(dtrucks_specs["truck_model"])

veh_cost_e = (1/260) * sum(
    float(scip.getVal(Flottenwahl[l, p])) * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L_e for p in P if l in L
)

veh_cost_d = (1/260) * sum(
    float(scip.getVal(Flottenwahl[l, p])) * (capex_veh[l] + opex_veh[l] - thg_e.get(l, 0))
    for l in L_d for p in P if l in L
)

veh_cost = veh_cost_e + veh_cost_d

# 2) Ladeinfrastruktur-Kosten
charger_cost = (1/260) * (
    c_capex + sum(
        float(scip.getVal(Auswahl_Charger[c, i])) * (capex_ch[c] + opex_ch[c])
        for c in C for i in I
    )
)

# 3) Speicher-Kosten (Capex + jährliche Abschreibung)
batt_kwh_val = float(scip.getVal(Batterie_kWh))
batt_kw_val = float(scip.getVal(Batterie_kW))
battery_cost = (1/260) * (
    (batt_kwh_val * c_capex_bat_kWh) +
    (batt_kw_val * c_capex_bat_kW) +
    (batt_kwh_val * 0.02 * c_capex_bat_kWh) +
    (batt_kw_val * 0.02 * c_capex_bat_kW)
)

# 4) Netzausbau-Kosten
grid_cost = (1/260) * float(scip.getVal(Ausbau_Netz)) * capex_grid_add

# 5) Stromkosten über alle Zeitintervalle mit weiterer Aufschlüsselung
electricity_cost_lkw = 0.0
electricity_cost_store_charge = 0.0
electricity_cost_store_discharge = 0.0
electricity_detail_rows = []
for t in T:
    load_lkw = sum(float(scip.getVal(Ladeleistung[p, t])) for p in P)
    load_store = float(scip.getVal(Laden_Speicher[t]))
    unload_store = float(scip.getVal(Entladen_Speicher[t]))

    cost_lkw = load_lkw * 0.25 * c_energy
    cost_store_charge = load_store * 0.25 * c_energy
    cost_store_discharge = -unload_store * 0.25 * c_energy  # Entladung entlastet das Netz

    electricity_cost_lkw += cost_lkw
    electricity_cost_store_charge += cost_store_charge
    electricity_cost_store_discharge += cost_store_discharge

    electricity_detail_rows.append({
        "time_interval": t,
        "load_lkw_kw": load_lkw,
        "load_store_kw": load_store,
        "unload_store_kw": unload_store,
        "cost_lkw_eur": cost_lkw,
        "cost_store_charge_eur": cost_store_charge,
        "cost_store_discharge_eur": cost_store_discharge,
        "cost_total_eur": cost_lkw + cost_store_charge + cost_store_discharge
    })

electricity_cost_total = electricity_cost_lkw + electricity_cost_store_charge + electricity_cost_store_discharge
df_electricity = pd.DataFrame(electricity_detail_rows)

# 6) Peak-Kosten (auf Basis P_peak; kann leicht von der implementierten Zielfunktion abweichen)
peak_cost = (1/260) * float(scip.getVal(P_peak)) * c_peak

# 7) Mautkosten nur für Diesel-Lkw (ActrosL)
toll_cost = 0.0
if 'ActrosL' in L:
    for r in R:
        for p in P:
            toll_cost += float(scip.getVal(Einsatz[r, 'ActrosL', p])) * dist_toll[r] * c_toll

# 8) Dieselkraftstoffkosten (ActrosL)
diesel_cost = 0.0
if 'ActrosL' in L:
    for r in R:
        for p in P:
            diesel_cost += float(scip.getVal(Einsatz[r, 'ActrosL', p])) * (dist_total[r] * cons_d.get('ActrosL', 0) / 100.0) * diesel

total_cost_decomposed = veh_cost + charger_cost + battery_cost + grid_cost + electricity_cost_total + peak_cost + toll_cost + diesel_cost
obj_val = float(scip.getObjVal())

cost_rows = [
    {"component": "Fahrzeuge_E", "cost_per_day": veh_cost_e},
    {"component": "Fahrzeuge_Diesel", "cost_per_day": veh_cost_d},
    {"component": "Fahrzeuge_Gesamt", "cost_per_day": veh_cost},
    {"component": "Ladeinfrastruktur", "cost_per_day": charger_cost},
    {"component": "Speicher", "cost_per_day": battery_cost},
    {"component": "Netzausbau", "cost_per_day": grid_cost},
    {"component": "Strom_LKW", "cost_per_day": electricity_cost_lkw},
    {"component": "Strom_Speicher_Laden", "cost_per_day": electricity_cost_store_charge},
    {"component": "Strom_Speicher_Entladen", "cost_per_day": electricity_cost_store_discharge},
    {"component": "Strom_Gesamt", "cost_per_day": electricity_cost_total},
    {"component": "Peak", "cost_per_day": peak_cost},
    {"component": "Maut_Diesel", "cost_per_day": toll_cost},
    {"component": "Diesel_Kraftstoff", "cost_per_day": diesel_cost},
    {"component": "Summe_Komponenten", "cost_per_day": total_cost_decomposed},
    {"component": "Objektwert_Solver", "cost_per_day": obj_val}
]
df_costs = pd.DataFrame(cost_rows)

# Schreiben in eine Excel-Datei mit mehreren Tabellenblättern
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    df_fleet.to_excel(writer, sheet_name="Flottenwahl", index=False)
    df_route_assign.to_excel(writer, sheet_name="Route_LKW", index=False)
    df_einsatz.to_excel(writer, sheet_name="Einsatz", index=False)
    df_charger.to_excel(writer, sheet_name="Charger_Wahl", index=False)
    df_soc.to_excel(writer, sheet_name="SOC_Fahrzeug", index=False)
    df_power.to_excel(writer, sheet_name="Ladeleistung", index=False)
    df_charger_assign.to_excel(writer, sheet_name="LKW_Charger_Zeit", index=False)
    df_storage.to_excel(writer, sheet_name="Speicher", index=False)
    df_scalars.to_excel(writer, sheet_name="Skalare", index=False)
    df_costs.to_excel(writer, sheet_name="Kostenaufschluss", index=False)
    df_electricity.to_excel(writer, sheet_name="Strom_Detail", index=False)

print(f"Optimierungsergebnisse wurden in '{excel_path}' gespeichert.")